In [34]:
import requests
import pandas as pd
import pymysql
import numpy as np
from datetime import datetime
import traceback2

In [49]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_solona_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            dt = row["date"].timestamp()
            symbol = row['symbol'] if row['symbol'] else ' '
            print(row['date'], row['name'], symbol, row['price'], row['p_pct_change'], row['tvl'], row['tags'])
            sql = """
            INSERT INTO `solonaprojects_data` (`timestamp_dt`, `name`, `token`, `price`, `p_pct_change_24h`, `tvl`, `tags`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            `price` = VALUES(price),
            `p_pct_change_24h` = VALUES(p_pct_change_24h),
            `tvl` = VALUES(tvl),
            `tags` = VALUES(tags)
            """
            cursor.execute(sql, (dt , row['name'], symbol, row['price'], row['p_pct_change'], row['tvl'], row['tags']))
        except Exception as e:
            print('exception')
            print(traceback2.format_exc())
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [50]:
insert_solona_data_df(df)

2021-05-04 00:00:00 Raydium RAY 14.700000000000000000 -6.11 958918903.70000000 DEX
2021-05-04 00:00:00 SerumDex SRM 11.810000000000000000 1.98 409816613.30945000 DEX
2021-05-04 00:00:00 Orca   0 0.00 55591380.36786700 AMM
2021-05-04 00:00:00 Oxygen OXY 3.133307573415765069 -2.48 37742643.92000000 Lending
2021-05-04 00:00:00 Serum Swap SRM 11.810000000000000000 1.98 35127954.34352000 AMM
2021-05-04 00:00:00 Mango   0 0.00 20940799.58819600 Perpetuals
2021-05-04 00:00:00 MATH Wallet MATH 1.995904173106646058 0.20 0 Wallet
2021-05-04 00:00:00 ChainLink LINK 40.950000000000000000 -0.09 0 Oracles
2021-05-04 00:00:00 The Graph GRT 1.540000000000000000 -6.30 0 Infrastructure
2021-05-04 00:00:00 Waves WAVES 39.540000000000000000 59.03 0 Cross-chain
2021-05-04 00:00:00 Ren REN 1.729443585780525502 4.74 0 Cross-chain
2021-05-04 00:00:00 Audius AUDIO 2.440000000000000000 -8.21 0 Music
2021-05-04 00:00:00 Solana SOL 46.068006182380216383 -4.66 0 
2021-05-04 00:00:00 Bonfida FIDA 3.4900000000000000

In [3]:
url = 'https://api.staked.xyz/apiSolana/getSolanaProjectTokensRank?sort=tvl&order=desc&limit=100&page=1'


In [4]:
response = requests.get(url)

In [8]:
data = response.json()['data']

In [10]:
len(data['list'])

25

In [41]:
items = []
for item in data['list']:
    items.append([item['title'], item['token'], item['price'], item['percentChange'], item['tvl'], ",".join(item['tags'])])

In [42]:
df = pd.DataFrame(items, columns=['name', 'symbol', 'price', 'p_pct_change', 'tvl', 'tags'])
now = datetime.now()
now = datetime(now.year, now.month, now.day)
df['date'] = now

In [43]:
df

,name,symbol,price,p_pct_change,tvl,tags,date
0,Raydium,RAY,14.700000000000000000,-6.11,958918903.70000000,DEX,2021-05-04
1,SerumDex,SRM,11.810000000000000000,1.98,409816613.30945000,DEX,2021-05-04
2,Orca,,0,0.00,55591380.36786700,AMM,2021-05-04
3,Oxygen,OXY,3.133307573415765069,-2.48,37742643.92000000,Lending,2021-05-04
4,Serum Swap,SRM,11.810000000000000000,1.98,35127954.34352000,AMM,2021-05-04
5,Mango,,0,0.00,20940799.58819600,Perpetuals,2021-05-04
6,MATH Wallet,MATH,1.995904173106646058,0.20,0,Wallet,2021-05-04
7,ChainLink,LINK,40.950000000000000000,-0.09,0,Oracles,2021-05-04
8,The Graph,GRT,1.540000000000000000,-6.30,0,Infrastructure,2021-05-04
9,Waves,WAVES,39.540000000000000000,59.03,0,Cross-chain,2021-05-04


In [33]:
insert_solona_data_df(df)

exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'
exception
'token'


In [25]:
df.to_csv('solona_data.csv')

In [24]:
df

,name,symbol,price,p_pct_change,tvl,tvl_short,tags,date
0,Raydium,RAY,14.700000000000000000,-6.11,958918903.70000000,958.92M,DEX,2021-05-04
1,SerumDex,SRM,11.810000000000000000,1.98,409816613.30945000,409.82M,DEX,2021-05-04
2,Orca,,0,0.00,55591380.36786700,55.59M,AMM,2021-05-04
3,Oxygen,OXY,3.133307573415765069,-2.48,37742643.92000000,37.74M,Lending,2021-05-04
4,Serum Swap,SRM,11.810000000000000000,1.98,35127954.34352000,35.13M,AMM,2021-05-04
5,Mango,,0,0.00,20940799.58819600,20.94M,Perpetuals,2021-05-04
6,MATH Wallet,MATH,1.995904173106646058,0.20,0,0,Wallet,2021-05-04
7,ChainLink,LINK,40.950000000000000000,-0.09,0,0,Oracles,2021-05-04
8,The Graph,GRT,1.540000000000000000,-6.30,0,0,Infrastructure,2021-05-04
9,Waves,WAVES,39.540000000000000000,59.03,0,0,Cross-chain,2021-05-04
